In [1]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs

Using TensorFlow backend.


#  0==angry
# 1== fear

In [2]:
clean_questions = pd.read_csv("emotion.csv")
clean_questions.columns=['label', 'sentences']
clean_questions.head()

,label,sentences
0,0,i feel bitter that it s not me
1,0,i feel the need to like her status even though...
2,0,i stayed home i'm feeling somewhat bitter
3,0,i sat there in frustration realizing that i wa...
4,0,i feel a little bitter towards the whole disease


In [3]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

clean_questions["tokens"] = clean_questions["sentences"].apply(tokenizer.tokenize)
clean_questions.head()

,label,sentences,tokens
0,0,i feel bitter that it s not me,"[i, feel, bitter, that, it, s, not, me]"
1,0,i feel the need to like her status even though...,"[i, feel, the, need, to, like, her, status, ev..."
2,0,i stayed home i'm feeling somewhat bitter,"[i, stayed, home, i, m, feeling, somewhat, bit..."
3,0,i sat there in frustration realizing that i wa...,"[i, sat, there, in, frustration, realizing, th..."
4,0,i feel a little bitter towards the whole disease,"[i, feel, a, little, bitter, towards, the, who..."


In [ ]:
import gensim

word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [ ]:
embeddings = get_word2vec_embeddings(word2vec, clean_questions)
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(embeddings, list_labels, 
                                                                                        test_size=0.2, random_state=40)

In [ ]:
clf_w2v = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', random_state=40)
clf_w2v.fit(X_train_word2vec, y_train_word2vec)
y_predicted_word2vec = clf_w2v.predict(X_test_word2vec)